In [2]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

# Load spaCy model
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [5]:
# python setup.py build_ext
import numpy as np
import json
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim

In [10]:
import nltk
nltk.download('stopwords')

In [6]:
stopwords = stopwords.words('english')

In [7]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
import pandas as pd
data = pd.read_excel('Drugs.xlsx')['New Tweet']
# get first 90 characters
print(data[0][0:90])
print(type(data))

lauren boebert calling abolishment dept education like chlamydia speaking out against dang
<class 'pandas.core.series.Series'>


In [21]:
# import en_core_web_sm
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    texts_out = []
    for text in texts:
 # Handle NaN values
        if pd.isna(text):
            text = ''
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return texts_out


lemmatized_texts = lemmatization(data)
print(lemmatized_texts[0][0:100])

call dept education chlamydia speak danger


In [22]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print(data_words[0][0:20])

['call', 'dept', 'education', 'chlamydia', 'speak', 'danger']


In [ ]:
#Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=7, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

# Function to make bigrams
def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

# Function to make trigrams
def make_trigram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigram(data_bigrams)

# Convert generators to lists
data_bigrams_list = list(data_bigrams)
data_bigrams_trigrams_list = list(data_bigrams_trigrams)

# Check if there are non-empty lists before printing
if data_bigrams_trigrams_list:
    print(data_bigrams_trigrams_list[0])
else:
    print("No non-empty bigrams and trigrams.")

In [67]:
# TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0][0:10])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words =[]
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for pid, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] #the words with tfidf score 0 will be missing
    
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]


In [68]:
lda_model2 = gensim.models.LdaMulticore(corpus, 
                                   num_topics = 5, 
                                   id2word = id2word,                                    
                                   passes = 50)

In [69]:
# For each topic, we will explore the words occuring in that topic and its relative weight

for idx, topic in lda_model2.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.034*"amoxicillin" + 0.014*"take" + 0.011*"lorazepam" + 0.008*"antibiotic" + 0.008*"make" + 0.007*"go" + 0.007*"get" + 0.007*"buy" + 0.006*"say" + 0.006*"patient"


Topic: 1 
Words: 0.025*"lorazepam" + 0.017*"get" + 0.015*"amoxicillin" + 0.012*"give" + 0.009*"use" + 0.008*"here" + 0.007*"infection" + 0.006*"treatment" + 0.006*"year" + 0.005*"make"


Topic: 2 
Words: 0.019*"amoxicillin" + 0.013*"lorazepam" + 0.013*"take" + 0.013*"get" + 0.011*"go" + 0.011*"day" + 0.010*"use" + 0.010*"nurofen" + 0.009*"covid" + 0.008*"week"


Topic: 3 
Words: 0.016*"get" + 0.016*"take" + 0.015*"amoxicillin" + 0.015*"so" + 0.011*"lorazepam" + 0.009*"sleep" + 0.009*"say" + 0.008*"give" + 0.008*"go" + 0.007*"drug"


Topic: 4 
Words: 0.036*"lorazepam" + 0.025*"take" + 0.013*"just" + 0.011*"need" + 0.010*"get" + 0.009*"work" + 0.009*"so" + 0.008*"now" + 0.008*"infection" + 0.007*"prescribe"




In [14]:
# Visualizing the model
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=5)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.201620 -0.013788       1        1  25.351666
2      0.073439  0.173693       2        1  24.047779
0     -0.173010 -0.106800       3        1  22.690616
3     -0.139383  0.120662       4        1  15.389592
4      0.037335 -0.173767       5        1  12.520346, topic_info=             Term       Freq       Total Category  logprob  loglift
30             so  40.000000   40.000000  Default   5.0000   5.0000
42           give  39.000000   39.000000  Default   4.0000   4.0000
55           take  92.000000   92.000000  Default   3.0000   3.0000
169          need  28.000000   28.000000  Default   2.0000   2.0000
224       patient  21.000000   21.000000  Default   1.0000   1.0000
224       patient  20.381401   21.009967   Topic1  -4.1695   1.3420
372           try   9.711146   10.310270   Topic1  -4.9108   1.3125
190         thank   9.659936   10.258345   Topic1  -4.9161   1.3122
353          hour   9.548544   10.151378   Topic1  -4.9277   1.3111
1220     diazepam   8.893043    9.486298   Topic1  -4.9988   1.3077
42           give  34.988469   39.676152   Topic1  -3.6291   1.2466
102           too  18.476169   22.334188   Topic1  -4.2676   1.1827
53            say  23.357111   31.962601   Topic1  -4.0332   1.0587
786          tell  13.348048   16.388440   Topic1  -4.5927   1.1671
55           take  31.756201   92.045071   Topic1  -3.7260   0.3081
1203    lorazepam  31.495613  124.727223   Topic1  -3.7342  -0.0040
16           time  16.135500   16.739116   Topic2  -4.3503   1.3884
202          very  11.884730   12.499163   Topic2  -4.6560   1.3747
524          also  10.527087   11.140815   Topic2  -4.7773   1.3685
365          much   8.784882    9.410972   Topic2  -4.9583   1.3563
346          test   7.979664    8.596548   Topic2  -5.0544   1.3507
30             so  30.266248   40.302449   Topic2  -3.7213   1.1387
547          pain  14.684031   17.608618   Topic2  -4.4445   1.2435
55           take  42.776369   92.045071   Topic2  -3.3753   0.6588
1203    lorazepam  51.060149  124.727223   Topic2  -3.1983   0.5320
105           day  16.874607   28.110743   Topic2  -4.3055   0.9148
29           just  18.066901   36.468706   Topic2  -4.2372   0.7228
76           dose  12.648057   13.248133   Topic3  -4.5357   1.4369
106     treatment  11.823656   12.442704   Topic3  -4.6031   1.4322
168          look  10.928130   11.524677   Topic3  -4.6819   1.4301
65           find   9.401387   10.002519   Topic3  -4.8323   1.4212
479         short   8.699698    9.334168   Topic3  -4.9099   1.4128
500          help  20.660694   25.774856   Topic3  -4.0450   1.2621
169          need  21.590029   28.272970   Topic3  -4.0010   1.2135
35           good  16.226340   20.294106   Topic3  -4.2866   1.2595
368        really  15.170000   18.519689   Topic3  -4.3539   1.2837
18    amoxicillin  21.409606   56.778988   Topic3  -4.0094   0.5079
1203    lorazepam  27.270985  124.727223   Topic3  -3.7674  -0.0371
55           take  17.226648   92.045071   Topic3  -4.2267  -0.1926
903        people   9.244388    9.905494   Topic4  -4.4609   1.8024
401          stop   8.308289    8.925076   Topic4  -4.5677   1.7999
618   side_effect   7.277158    7.888146   Topic4  -4.7002   1.7909
1024         link   6.936210    7.538638   Topic4  -4.7482   1.7882
340          hope   6.621411    7.271697   Topic4  -4.7946   1.7778
115           use  12.819500   37.659296   Topic4  -4.1340   0.7939
1203    lorazepam  14.668447  124.727223   Topic4  -3.9992  -0.2690
30             so   9.568443   40.302449   Topic4  -4.4265   0.4335
878         leave   6.198977    6.809442   Topic5  -4.6542   1.9839
696         child   5.657829    6.280361   Topic5  -4.7456   1.9734
334          shit   5.010341    5.621835   Topic5  -4.8671   1.9627
804          talk   5.313106    5.967346   Topic5  -4.8084   1.9617
1091          far   4.568786    5.224245   

In [15]:
# Amoxicillin LDA model

In [16]:
import pandas as pd
am_data = pd.read_excel('Amoxicillin.xlsx')['New Tweet']
# get first 90 characters
print(am_data[0][0:90])
print(type(am_data))

lauren boebert calling abolishment dept education like chlamydia speaking out against dang
<class 'pandas.core.series.Series'>


In [17]:
# import en_core_web_sm
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    texts_out = []
    for text in texts:
 # Handle NaN values
        if pd.isna(text):
            text = ''
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return texts_out


lemmatized_texts = lemmatization(am_data)
print(lemmatized_texts[0][0:100])

call dept education chlamydia speak danger


In [18]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

am_data_words = gen_words(lemmatized_texts)
print(am_data_words[0][0:20])

['call', 'dept', 'education', 'chlamydia', 'speak', 'danger']


In [19]:
#Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(am_data_words, min_count=7, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[am_data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

# Function to make bigrams
def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

# Function to make trigrams
def make_trigram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams = make_bigrams(am_data_words)
data_bigrams_trigrams = make_trigram(data_bigrams)

# Convert generators to lists
data_bigrams_list = list(data_bigrams)
data_bigrams_trigrams_list = list(data_bigrams_trigrams)

# Check if there are non-empty lists before printing
if data_bigrams_trigrams_list:
    print(data_bigrams_trigrams_list[0])
else:
    print("No non-empty bigrams and trigrams.")


['call', 'dept', 'education', 'chlamydia', 'speak', 'danger']


In [20]:
# TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0][0:10])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words =[]
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for pid, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] #the words with tfidf score 0 will be missing
    
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]


In [21]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto')

In [22]:
# Visualizing the model
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=3)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.007970  0.138985       1        1  47.993602
2      0.101136 -0.069547       2        1  26.408801
0     -0.109106 -0.069438       3        1  25.597598, topic_info=             Term       Freq      Total Category  logprob  loglift
55           take  27.000000  27.000000  Default   3.0000   3.0000
42           give  27.000000  27.000000  Default   2.0000   2.0000
115           use  20.000000  20.000000  Default   1.0000   1.0000
55           take  27.134200  27.580548   Topic1  -3.8691   0.7178
21      prescribe  10.254748  10.669496   Topic1  -4.8421   0.6945
30             so   8.881303   9.294193   Topic1  -4.9859   0.6887
42           give  25.950505  27.536128   Topic1  -3.9137   0.6748
17           week  16.276115  17.350338   Topic1  -4.3802   0.6702
18    amoxicillin  62.308663  84.536128   Topic1  -3.0378   0.4290
28            get  26.024616  39.836011   Topic1  -3.9108   0.3084
1026  prophylaxis   6.943683   7.405648   Topic2  -4.6347   1.2671
1022       decade   4.250660   4.712628   Topic2  -5.1254   1.2283
1011      various   4.249594   4.712902   Topic2  -5.1257   1.2280
115           use  11.999831  20.903869   Topic2  -4.0876   0.7764
34     antibiotic  10.494339  19.518409   Topic2  -4.2217   0.7110
18    amoxicillin  14.958702  84.536128   Topic2  -3.8672  -0.4004
679        cancer   4.811894   5.345069   Topic3  -4.9702   1.2576
114         thing   5.293120   5.886707   Topic3  -4.8749   1.2564
169          need   3.991868   4.490661   Topic3  -5.1570   1.2449
264          drug   7.053216  11.254591   Topic3  -4.5878   0.8954
28            get   8.187261  39.836011   Topic3  -4.4387  -0.2195
115           use   6.483160  20.903869   Topic3  -4.6721   0.1919
18    amoxicillin   7.268762  84.536128   Topic3  -4.5577  -1.0909, token_table=      Topic      Freq         Term
term                              
18        1  0.733414  amoxicillin
18        2  0.177439  amoxicillin
18        3  0.082805  amoxicillin
34        1  0.204935   antibiotic
34        2  0.512337   antibiotic
34        3  0.256168   antibiotic
679       3  0.935442       cancer
1022      2  0.848783       decade
264       1  0.088853         drug
264       2  0.266558         drug
264       3  0.621968         drug
28        1  0.652676          get
28        2  0.150617          get
28        3  0.200823          get
42        1  0.944214         give
42        3  0.036316         give
169       3  0.890737         need
21        1  0.937251    prescribe
1026      2  0.945225  prophylaxis
30        1  0.968347           so
55        1  0.978951         take
114       3  0.849371        thing
115       1  0.095676          use
115       2  0.574056          use
115       3  0.287028          use
1011      2  0.848734      various
17        1  0.922172         week
17        2  0.057636         week, R=3, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [23]:
# Nurofen LDA model

In [24]:
import pandas as pd
nu_data = pd.read_excel('Nurofen.xlsx')['New Tweet']
# get first 90 characters
print(nu_data[0][0:90])
print(type(nu_data))

looking box nurofen like what if took too many
<class 'pandas.core.series.Series'>


In [25]:
# import en_core_web_sm
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    texts_out = []
    for text in texts:
 # Handle NaN values
        if pd.isna(text):
            text = ''
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return texts_out


lemmatized_texts = lemmatization(nu_data)
print(lemmatized_texts[0][0:100])

look box nurofen take too many


In [26]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

nu_data_words = gen_words(lemmatized_texts)
print(nu_data_words[0][0:20])

['look', 'box', 'nurofen', 'take', 'too', 'many']


In [27]:
# Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(nu_data_words, min_count=7, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[nu_data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

# Function to make bigrams
def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

# Function to make trigrams
def make_trigram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams = make_bigrams(nu_data_words)
data_bigrams_trigrams = make_trigram(data_bigrams)

# Convert generators to lists
data_bigrams_list = list(data_bigrams)
data_bigrams_trigrams_list = list(data_bigrams_trigrams)

# Check if there are non-empty lists before printing
if data_bigrams_trigrams_list:
    print(data_bigrams_trigrams_list[0])
else:
    print("No non-empty bigrams and trigrams.")

['look', 'box', 'nurofen', 'take', 'too', 'many']


In [28]:
# TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0][0:10])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words =[]
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for pid, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] #the words with tfidf score 0 will be missing
    
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]


In [29]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto')

In [30]:
# Visualizing the model
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=3)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.079539  0.035063       1        1  37.150579
0     -0.007467 -0.093986       2        1  32.542460
2     -0.072072  0.058923       3        1  30.306961, topic_info=        Term       Freq      Total Category  logprob  loglift
60       say   6.000000   6.000000  Default   3.0000   3.0000
3    nurofen  17.000000  17.000000  Default   2.0000   2.0000
61        so   5.000000   5.000000  Default   1.0000   1.0000
107     time   2.116868   2.533569   Topic1  -4.5327   0.8105
193     even   1.493579   1.894236   Topic1  -4.8815   0.7526
134     half   1.491549   1.894111   Topic1  -4.8829   0.7513
26      help   2.772632   3.768613   Topic1  -4.2629   0.6833
3    nurofen   7.252475  17.940395   Topic1  -3.3013   0.0845
118     pain   3.415274   6.837893   Topic1  -4.0544   0.2960
4       take   4.048908   9.893511   Topic1  -3.8842   0.0968
275    clove   1.416797   1.828270   Topic2  -4.8019   0.8677
190     same   1.416678   1.828271   Topic2  -4.8019   0.8676
99      make   1.416514   1.828275   Topic2  -4.8021   0.8675
9       give   2.629759   3.682790   Topic2  -4.1834   0.7858
3    nurofen   8.753769  17.940395   Topic2  -2.9808   0.4051
16      just   3.852682   7.319619   Topic2  -3.8015   0.4808
4       take   4.459953   9.893511   Topic2  -3.6551   0.3259
58      need   1.976874   2.396040   Topic3  -4.3976   1.0015
94     right   1.971200   2.396506   Topic3  -4.4004   0.9984
199    budge   1.385718   1.801873   Topic3  -4.7529   0.9312
60       say   4.348215   6.019915   Topic3  -3.6093   0.8685
73       day   2.571768   3.630173   Topic3  -4.1345   0.8491
61        so   3.121227   5.506649   Topic3  -3.9409   0.6261
103      bad   2.572932   4.237254   Topic3  -4.1340   0.6949, token_table=      Topic      Freq     Term
term                          
103       1  0.236002      bad
103       2  0.236002      bad
103       3  0.708006      bad
199       3  0.554978    budge
275       2  0.546965    clove
73        1  0.275469      day
73        3  0.826407      day
193       1  0.527917     even
9         1  0.271533     give
9         2  0.814600     give
134       1  0.527952     half
26        1  0.796049     help
26        3  0.265350     help
16        1  0.136619     just
16        2  0.546477     just
16        3  0.273238     just
99        2  0.546964     make
58        3  0.834711     need
3         1  0.390181  nurofen
3         2  0.501661  nurofen
3         3  0.111480  nurofen
118       1  0.438732     pain
118       2  0.438732     pain
118       3  0.146244     pain
94        3  0.834548    right
190       2  0.546965     same
60        1  0.166115      say
60        2  0.166115      say
60        3  0.664461      say
61        1  0.363197       so
61        3  0.544796       so
4         1  0.404305     take
4         2  0.404305     take
4         3  0.101076     take
107       1  0.789400     time, R=3, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [31]:
# Lorazepam LDA model

In [32]:
import pandas as pd
lo_data = pd.read_excel('Lorazepam.xlsx')['New Tweet']
# get first 90 characters
print(lo_data[0][0:90])
print(type(lo_data))

lorazepam stimulates il6 production  associated  poor survival outcomes  pancreatic cancer
<class 'pandas.core.series.Series'>


In [33]:
# import en_core_web_sm
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    texts_out = []
    for text in texts:
 # Handle NaN values
        if pd.isna(text):
            text = ''
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return texts_out


lemmatized_texts = lemmatization(lo_data)
print(lemmatized_texts[0][0:100])

lorazepam stimulate il6 production associate poor survival outcome pancreatic cancer clinical cancer


In [34]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

lo_data_words = gen_words(lemmatized_texts)
print(lo_data_words[0][0:20])

['lorazepam', 'stimulate', 'il', 'production', 'associate', 'poor', 'survival', 'outcome', 'pancreatic', 'cancer', 'clinical', 'cancer', 'research', 'cancer', 'research']


In [35]:
# Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(lo_data_words, min_count=7, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[lo_data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

# Function to make bigrams
def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

# Function to make trigrams
def make_trigram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams = make_bigrams(lo_data_words)
data_bigrams_trigrams = make_trigram(data_bigrams)

# Convert generators to lists
data_bigrams_list = list(data_bigrams)
data_bigrams_trigrams_list = list(data_bigrams_trigrams)

# Check if there are non-empty lists before printing
if data_bigrams_trigrams_list:
    print(data_bigrams_trigrams_list[0])
else:
    print("No non-empty bigrams and trigrams.")

['lorazepam', 'stimulate', 'il', 'production', 'associate', 'poor', 'survival', 'outcome', 'pancreatic_cancer', 'clinical', 'cancer', 'research', 'cancer', 'research']


In [39]:
# TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0][0:10])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words =[]
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for pid, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(item)
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] #the words with tfidf score 0 will be missing
    
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2)]


In [37]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto')

In [54]:
# Assuming your LDA model is stored in lda_model
# and the corpus and dictionary are defined as corpus and id2word

# Get the top N terms for each topic
N = 3  # Adjust the number of terms as needed
top_terms_per_topic = {i: [id2word[word_id] for word_id, prob in lda_model.get_topic_terms(i, topn=N)] for i in range(lda_model.num_topics)}

# Visualize the model
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, sort_topics=True, mds='mmds', R=3)

# Customize the hover tooltip
for i in range(lda_model.num_topics):
    terms_str = ', '.join(top_terms_per_topic[i])
    vis.topic_info.loc[i, 'Terms'] = f'Top 3 Terms: {terms_str}'

# Save the visualization to an HTML file
pyLDAvis.save_html(vis, 'lda_visualization.html')
